Code for introducing custom CV forces 

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np

In [2]:
prot = md.load_pdb('custom-test/test-dimer/dimer.pdb')

In [3]:
topology = prot.topology #need to delete TER records before NHE

In [4]:
g1 = [atom.index for atom in topology.chain(0).atoms]

In [5]:
g2 = [atom.index for atom in topology.chain(1).atoms]

In [32]:
prmtop = AmberPrmtopFile('custom-test/test-dimer/dimer.prmtop')
inpcrd = AmberInpcrdFile('custom-test/test-dimer/dimer.inpcrd')
pdb = PDBFile('custom-test/test-dimer/dimer.pdb')

system = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1*nanometer,
        constraints=HBonds, implicitSolvent=OBC2)

g1_positions = [pdb.positions[index] for index in g1]
g2_positions = [pdb.positions[index] for index in g2]

r = openmm.CustomCentroidBondForce(2, 'distance(g1, g2)')
r.addGroup(g1)
r.addGroup(g2)
r.addBond([0,1])

cvforce = openmm.CustomCVForce('r')
cvforce.addCollectiveVariable('r', r)
system.addForce(cvforce)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.002*picoseconds)

platform = Platform.getPlatformByName('CUDA')

simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('custom-test/test-dimer/output-3.dcd', 1000))
#simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
#        potentialEnergy=True, temperature=True))


simulation.step(100000)
print(cvforce.getCollectiveVariableValues(simulation.context))

(0.9757857918739319,)


In [9]:
#prmtop = AmberPrmtopFile('custom-test/dimer.prmtop')
#inpcrd = AmberInpcrdFile('custom-test/dimer.inpcrd')
#pdb = PDBFile('custom-test/dimer.pdb')

#system = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1*nanometer,
#        constraints=HBonds, implicitSolvent=OBC2)

#integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.002*picoseconds)

#platform = Platform.getPlatformByName('CUDA')

#simulation = Simulation(prmtop.topology, system, integrator, platform)
#simulation.context.setPositions(inpcrd.positions)
#simulation.minimizeEnergy()
#simulation.reporters.append(DCDReporter('custom-test/simple.dcd', 1000))
#simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
#        potentialEnergy=True, temperature=True))
#simulation.step(100000)

Scale up for tetramer

In [2]:
prot = md.load_pdb('custom-test/test-tetramer/tetramer.pdb')
topology = prot.topology
topology

<mdtraj.Topology with 4 chains, 128 residues, 1916 atoms, 1908 bonds at 0x7faed843ba30>

In [72]:
g1 = [atom.index for atom in topology.chain(0).atoms]
g2 = [atom.index for atom in topology.chain(1).atoms]
g3 = [atom.index for atom in topology.chain(2).atoms]
g4 = [atom.index for atom in topology.chain(3).atoms]

In [ ]:
g1_positions = [pdb.positions[index] for index in g1]
g2_positions = [pdb.positions[index] for index in g2]
g3_positions = [pdb.positions[index] for index in g3]
g4_positions = [pdb.positions[index] for index in g4]

#d12 = np.linalg.norm(np.mean(np.asarray(g1_positions), axis=0) - np.mean(np.asarray(g2_positions), axis=0))

In [86]:
prmtop = AmberPrmtopFile('custom-test/test-tetramer/tetramer.prmtop')
inpcrd = AmberInpcrdFile('custom-test/test-tetramer/tetramer.inpcrd')
pdb = PDBFile('custom-test/test-tetramer/tetramer.pdb')

system = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1*nanometer,
        constraints=HBonds, implicitSolvent=OBC2)

expr = 'distance(g1,g2) + distance(g1,g3) + distance(g1,g4) + distance(g2,g3)+ distance(g2,g4) +distance(g3,g4)'

force = openmm.CustomCentroidBondForce(4, expr)
force.addGroup(g1)
force.addGroup(g2)
force.addGroup(g3)
force.addGroup(g4)
ainds=[0,1,2,3]
force.addBond(ainds)


cvforce = openmm.CustomCVForce('r')
cvforce.addCollectiveVariable('r', force) 
system.addForce(cvforce)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.002*picoseconds)

platform = Platform.getPlatformByName('CUDA')

simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('custom-test/test-tetramer/output-2.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))


simulation.step(10000)
print(cvforce.getCollectiveVariableValues(simulation.context))

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-11335.005934715271,260.9061401170132
2000,-10540.799880981445,302.39191526349674
3000,-10342.21229171753,293.7682363136886
4000,-10443.635043144226,300.7833187289228
5000,-10298.682028770447,296.68632260892394
6000,-10523.131711959839,292.9011936436644
7000,-10336.002591133118,291.3093203418392
8000,-10232.676971435547,304.2400623958286
9000,-10415.02967453003,304.0774909955328
10000,-10532.85629940033,295.6784032639435
(10.05385684967041,)
<openmm.openmm.CustomCentroidBondForce; proxy of <Swig Object of type 'OpenMM::CustomCentroidBondForce *' at 0x7fae7dcc9d50> >
